In [12]:
import torch
from src.DEFAULTS import normalZ
from src.losses import MMD_Loss
from src.losses import avg_pinball




Y = torch.randn(100)

taus = 0.7 * torch.ones(100)
preds = normalZ.ppf(0.7) * torch.ones(100)

Y_inpu = torch.stack([preds, taus], dim = 1)

avg_pinball(Y_inpu, Y)



tensor(0.3907)

In [15]:
normalZ.ppf(0.9)

1.2815515655446004

In [4]:
preds = normalZ.ppf(0.2) * torch.ones(100)

Y_inpu = torch.stack([preds, taus], dim = 1)

avg_pinball(Y_inpu, Y)

tensor(0.6550)

In [6]:
import numpy as np

q_list = np.array([0.2, 0.8])

torch.Tensor(q_list).view(-1, 1).repeat(1, 10).view(-1)

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000,
        0.2000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000,
        0.8000, 0.8000])

In [7]:
y = torch.Tensor([
    [0.2, 0.4],
    [0.2, 0.6],
    [0.2, 0.8]
])

torch.permute(y, (1, 0)).reshape(-1,)

tensor([0.2000, 0.2000, 0.2000, 0.4000, 0.6000, 0.8000])

In [9]:
out_ten = torch.Tensor([
    [3, 0.1],
    [4, 0.1],
    [5, 0.2],
    [6, 0.2],
    [7, 0.3],
    [6, 0.3]
])
num_pts = 2
num_q = 3



out_ten[:,1].view(num_q, num_pts)[:, 0]


tensor([0.1000, 0.2000, 0.3000])

In [11]:
Y = torch.Tensor([3,1])
Y.repeat(num_q).reshape(num_q, num_pts)

tensor([[3., 1.],
        [3., 1.],
        [3., 1.]])